# Backtesting TrailingStrategy

backtesting 的策略設置裡雖然有 stop-loss 和 take-profit，但都是針對下單那一刻的價格設置固定值，想要動態改變就需要用到 TailingStrategy

In [1]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

import pandas as pd
import pandas_ta as ta
import yfinance as yf

/Users/kimfaicheang/workspace/snowball/venv/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df = yf.download("^SPX", period="60d", interval="5m")

[*********************100%%**********************]  1 of 1 completed


### 止蝕 / 止盈 基本設置

sma 金叉死叉，只做多不做空，全憑止蝕/止盈平倉

In [3]:
class ST(Strategy):
    def init(self):
        close = pd.Series(self.data.Close)
        self.sma_short = self.I(ta.sma, close, 9)
        self.sma_long = self.I(ta.sma, close, 17)

    def next(self):
        price = self.data.Close[-1]

        if self.position:
            pass
        else:
            # sl = stop loss, tp = take profits
            if crossover(self.sma_short, self.sma_long):
                self.buy(size=1, sl=price * 0.99, tp=price * 1.02)
            # elif crossover(self.sma_long, self.sma_short):
            #     self.sell(size=1, sl=price * 1.01, tp=price * 0.98)

# might want trade_on_close = True to deal with gapping for sl and tp
bt = Backtest(df, ST, commission=0.0002)
bt.run()

Start                     2023-12-05 09:30...
End                       2024-03-01 11:15...
Duration                     87 days 01:45:00
Exposure Time [%]                   95.480104
Equity Final [$]                  10420.57068
Equity Peak [$]                  10424.050661
Return [%]                           4.205707
Buy & Hold Return [%]               12.376854
Return (Ann.) [%]                   18.833959
Volatility (Ann.) [%]                5.724781
Sharpe Ratio                           3.2899
Sortino Ratio                        6.445517
Calmar Ratio                        12.961271
Max. Drawdown [%]                   -1.453095
Avg. Drawdown [%]                   -0.085672
Max. Drawdown Duration       34 days 20:35:00
Avg. Drawdown Duration        0 days 18:30:00
# Trades                                    8
Win Rate [%]                             75.0
Best Trade [%]                       1.979833
Worst Trade [%]                     -1.020009
Avg. Trade [%]                    

### 動態止蝕 / 止盈

baacktesting 有一個 TailingStrategy class, 這裡參考原版重寫一個

In [4]:
class CuzTrailingStrategy(Strategy):
    __sl_pct = 0.01

    def init(self):
        super().init()

    def set_trailing_sl(self, sl_amount: float = 6):
        self.__sl_amount = sl_amount

    def next(self):
        super().next()
        # Can't use index=-1 because self.__atr is not an Indicator type
        index = len(self.data)-1

        for trade in self.trades:
            if trade.is_long:
                trade.sl = max(trade.sl or -np.inf,
                               self.data.Close[index] * (1 - self.__sl_pct))
            else:
                trade.sl = min(trade.sl or np.inf,
                               self.data.Close[index] * (1 + self.__sl_pct))

In [5]:
class ST2(CuzTrailingStrategy):
    def init(self):
        super().init() # 初始化 TailingStrategy 
        super().set_trailing_sl(0.01)
        
        close = pd.Series(self.data.Close)
        self.sma_short = self.I(ta.sma, close, 9)
        self.sma_long = self.I(ta.sma, close, 17)

    def next(self):
        super().next() # 每個 loop 都重新計算 sl
        price = self.data.Close[-1]

        if self.position:
            pass
        else:
            # 這裡的 sl 只是初始值，每個 loop 會根據當前 close 修改
            if crossover(self.sma_short, self.sma_long):
                self.buy(size=1, sl=price * 0.99, tp=price * 1.02)
            # elif crossover(self.sma_long, self.sma_short):
            #     self.sell(size=1, sl=price * 1.01, tp=price * 0.98)

# might want trade_on_close = True to deal with gapping for sl and tp
bt = Backtest(df, ST2, commission=0.0002)
bt.run()

Start                     2023-12-05 09:30...
End                       2024-03-01 11:15...
Duration                     87 days 01:45:00
Exposure Time [%]                   91.652249
Equity Final [$]                 10436.608462
Equity Peak [$]                  10440.088443
Return [%]                           4.366085
Buy & Hold Return [%]               12.376854
Return (Ann.) [%]                   19.603997
Volatility (Ann.) [%]                5.952485
Sharpe Ratio                         3.293414
Sortino Ratio                        6.329373
Calmar Ratio                        17.144665
Max. Drawdown [%]                   -1.143446
Avg. Drawdown [%]                   -0.091642
Max. Drawdown Duration       25 days 01:10:00
Avg. Drawdown Duration        0 days 17:52:00
# Trades                                   19
Win Rate [%]                        63.157895
Best Trade [%]                        1.98338
Worst Trade [%]                     -1.514155
Avg. Trade [%]                    